In [1]:
import os
import csv
import random
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [2]:
cs_results = []
with open("../dataset/lancers_task.csv", encoding="utf-8", newline="") as f:
    for cols in csv.reader(f, delimiter=","):
        cs_results.append(cols)
#delete header
cs_results.pop(0)

['taskFormSelectBox2',
 'taskFormSelectBox3',
 'taskFormSelectBox4',
 'taskFormSelectBox5',
 'taskFormSelectBox6',
 'taskFormSelectBox7',
 'taskFormSelectBox9',
 'taskFormSelectBox10',
 'taskFormSelectBox11',
 'taskFormSelectBox12',
 'taskFormSelectBox13',
 'taskFormSelectBox14',
 'taskFormSelectBox16',
 'taskFormSelectBox17',
 'taskFormSelectBox18',
 'taskFormSelectBox19',
 'taskFormSelectBox20',
 'taskFormSelectBox21',
 'taskFormSelectBox23',
 'taskFormSelectBox24',
 'taskFormSelectBox25',
 'taskFormSelectBox26',
 'taskFormSelectBox27',
 'taskFormSelectBox28',
 'taskFormSelectBox30',
 'taskFormSelectBox31',
 'taskFormSelectBox32',
 'taskFormSelectBox33',
 'taskFormSelectBox34',
 'taskFormSelectBox35',
 'taskFormSelectBox37',
 'taskFormSelectBox38',
 'taskFormSelectBox39',
 'taskFormSelectBox40',
 'taskFormSelectBox41',
 'taskFormSelectBox42',
 'taskFormTextArea43']

In [3]:
def convert_name(table):
    result = []
    activity_class_map = {
        "就寝・起床": "BedTimeSleep",
        "飲食": "EatingDrinking",
        "飲食準備": "FoodPreparation",
        "整理・整頓": "HouseArrangement",
        "清掃・洗浄": "HouseCleaning",
        "衛生": "HygieneStyling",
        "娯楽": "Leisure",
        "仕事・学業": "Work",
        "身体活動": "PhysicalActivity",
        "その他": "Other"
    }
    for line in table:
        new_line = []
        for x in line:
            new_line.append(activity_class_map[x])
        result.append(new_line)
    return result

In [4]:
def add_start_end(sequence):
    tmp = ['start']
    tmp.extend(sequence)
    tmp.extend(['end'])
    return tmp

In [5]:
weekdays = []
holidays = []
for x in cs_results:
    weekdays.append(x[0:18])
    holidays.append(x[18:36])
weekdays =  convert_name(weekdays)
holidays =  convert_name(holidays)

In [6]:
weekdays_morning = []
weekdays_afternoon = []
weekdays_evening = []
holidays_morning = []
holidays_afternoon = []
holidays_evening = []
for x in weekdays:
   weekdays_morning.append(add_start_end(x[0:6]))
   weekdays_afternoon.append(add_start_end(x[6:12]))
   weekdays_evening.append(add_start_end(x[12:18]))

for x in holidays:
   holidays_morning.append(add_start_end(x[0:6]))
   holidays_afternoon.append(add_start_end(x[6:12]))
   holidays_evening.append(add_start_end(x[12:18]))

In [7]:
sequence_list = weekdays_morning
sequence_list.extend(weekdays_afternoon)
sequence_list.extend(weekdays_evening)
sequence_list.extend(holidays_morning)
sequence_list.extend(holidays_afternoon)
sequence_list.extend(holidays_evening)

In [8]:
# sequence_list = []
# for x in weekdays:
#     sequence_list.append(add_start_end(x))
# for x in holidays:
#     sequence_list.append(add_start_end(x))

In [9]:
sequence_list

[['start',
  'BedTimeSleep',
  'HygieneStyling',
  'Leisure',
  'FoodPreparation',
  'EatingDrinking',
  'Work',
  'Leisure',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'Work',
  'HygieneStyling',
  'Work',
  'Leisure',
  'Leisure',
  'FoodPreparation',
  'EatingDrinking',
  'BedTimeSleep',
  'end'],
 ['start',
  'BedTimeSleep',
  'FoodPreparation',
  'EatingDrinking',
  'HygieneStyling',
  'HouseArrangement',
  'Leisure',
  'HouseArrangement',
  'HouseCleaning',
  'Leisure',
  'BedTimeSleep',
  'PhysicalActivity',
  'BedTimeSleep',
  'FoodPreparation',
  'EatingDrinking',
  'PhysicalActivity',
  'HygieneStyling',
  'Leisure',
  'BedTimeSleep',
  'end'],
 ['start',
  'BedTimeSleep',
  'HouseArrangement',
  'EatingDrinking',
  'HygieneStyling',
  'BedTimeSleep',
  'Other',
  'FoodPreparation',
  'EatingDrinking',
  'HygieneStyling',
  'BedTimeSleep',
  'HygieneStyling',
  'Work',
  'Work',
  'FoodPreparation',
  'EatingDrinking',
  'HouseCleaning',
  'Leisure',
  'Be

In [10]:
def create_ngram(sequence_list):    
    ngram = {}
    for seq in sequence_list:
        for i in range(19):
            if seq[i] in ngram:
                values = ngram[seq[i]]
                if seq[i+1] in values.keys():
                    values[seq[i+1]] += 1
                else:
                    values[seq[i+1]] = 1
                ngram[seq[i]] = values
            else:
                tmp = {}
                tmp[seq[i+1]] = 1
                ngram[seq[i]] = tmp
    return ngram

In [11]:
def create_transition_probability(ngram):    
    transition_probability = {}
    for current_activity in ngram:
        next_activities = ngram[current_activity]
        num = 0
        for na_key in next_activities:
            num += next_activities[na_key]
        probability = {}
        for na_key in next_activities:
            probability[na_key] = next_activities[na_key] / num
        transition_probability[current_activity] =probability
    return transition_probability

In [12]:
def markov_chain(transition_probability):    
    current_activity = "start"
    activity_list = []
    for i in range(6):
        next_candidates = {}
        next_candidates = transition_probability[current_activity]

        keys = []
        values = []
        for x in next_candidates:
            keys.append(x)
            values.append(next_candidates[x])
        next_activity = random.choices(
            population=keys,
            weights=values
        )
        current_activity = next_activity[0]
        if current_activity == "end":
            break
        activity_list.append(current_activity)
    return activity_list

In [13]:
ngram = create_ngram(sequence_list)
transition_probability = create_transition_probability(ngram)
routine_list = []
i=0
while i < 20:
    mc = markov_chain(transition_probability)
    if len(mc) == 6:
        routine_list.append(mc)
        i += 1

In [14]:
routine_list

[['BedTimeSleep',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'HouseArrangement',
  'HouseCleaning',
  'Leisure',
  'Leisure',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'HouseCleaning',
  'HygieneStyling',
  'BedTimeSleep',
  'BedTimeSleep',
  'HygieneStyling',
  'Work',
  'BedTimeSleep'],
 ['BedTimeSleep',
  'FoodPreparation',
  'EatingDrinking',
  'Leisure',
  'HygieneStyling',
  'Work',
  'Work',
  'EatingDrinking',
  'HouseCleaning',
  'HygieneStyling',
  'HouseCleaning',
  'HouseCleaning',
  'PhysicalActivity',
  'BedTimeSleep',
  'HouseArrangement',
  'Leisure',
  'HouseArrangement',
  'Leisure'],
 ['BedTimeSleep',
  'PhysicalActivity',
  'FoodPreparation',
  'EatingDrinking',
  'HouseCleaning',
  'Leisure',
  'HygieneStyling',
  'FoodPreparation',
  'EatingDrinking',
  'Work',
  'HouseCleaning',
  'HygieneStyling',
  'Work',
  'Work',
  'FoodPreparation',
  'EatingDrinking',
  'HouseArrangement',
  'EatingDrinking'],
 ['BedTimeSleep',
  'L

In [15]:
rdf_g = rdflib.Graph()
rdf_g.parse("../ontology/vh2kg_ontology.ttl", format="ttl")

<Graph identifier=Na464e3b46b494987a2801371be980459 (<class 'rdflib.graph.Graph'>)>

In [16]:
def get_activity_from_ontology(activity_type):
    result = None
    queryString = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX : <http://www.owl-ontologies.com/VirtualHome.owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select distinct ?label where { 
            ?activity a :%s ;
                rdfs:label ?label .
         } ORDER BY RAND() limit 1 
    """
    sparql = SPARQLWrapper("http://localhost:7200/repositories/VirtualHome2Kg_r1_augmented")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials('admin', '')
    sparql.addParameter('infer', 'true')
    

    sparql.setQuery(queryString % activity_type)
    sparql.setReturnFormat(JSON)

    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        if len(bindings) > 0:
            result = bindings[0]
    except  Exception as e:
        print(e.args)
    return result

In [17]:
new_routine_list = []
for routine in routine_list:
    new_routine = []
    for activity in routine:
        binding = get_activity_from_ontology(activity)
        if binding is not None:
            new_routine.append(binding["label"]["value"])
    new_routine_list.append(new_routine)

In [18]:
new_routine_list

[['Go to sleep',
  'Wash teeth',
  'Make coffee',
  'Have snack',
  'Wash dishes',
  'Wash dirty dishes',
  'Watch youtube',
  'Watch movie',
  'Wash teeth',
  'Keep an eye on stove as something is cooking',
  'Have snack',
  'Clean',
  'Wash teeth',
  'Go to sleep',
  'Sleep',
  'Wash teeth',
  'Write an email',
  'Take nap'],
 ['Sleep',
  'Cook some food',
  'Have snack',
  'Watch movie',
  'Wash teeth',
  'Write an email',
  'Write an email',
  'Have snack',
  'Wipe down sink',
  'Wash teeth',
  'Wipe down sink',
  'Organize closet',
  'Sleep',
  'Wash dishes',
  'Watch movie',
  'Wash dishes with dishwasher',
  'Watch movie'],
 ['Go to sleep',
  'Cook some food',
  'Have snack',
  'Wash dirty dishes',
  'Surf net',
  'Wash teeth',
  'Make toast',
  'Have snack',
  'Write an email',
  'Wash sink',
  'Wash teeth',
  'Write an email',
  'Write an email',
  'Make coffee',
  'Have snack',
  'Wash dishes',
  'Have snack'],
 ['Take nap',
  'Surf net',
  'Wash teeth',
  'Watch youtube',
  

In [19]:
def export(routine_list):
     with open("graph_state_list/episode_list7_scene4.txt", 'w') as f:
            for routine in routine_list:
                f.write("%s\n" % ','.join(routine))

In [20]:
export(new_routine_list)